In [1]:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    from io import StringIO
    import time

    def get_day_price(code, max_pages=519):
        base_url = f"https://finance.naver.com/item/sise_day.naver?code={code}"
        headers = {"User-Agent": "Mozilla/5.0"}
        df_total = pd.DataFrame()

        for page in range(1, max_pages + 1):
            url = f"{base_url}&page={page}"
            res = requests.get(url, headers=headers)
            if res.status_code != 200:
                print(f"페이지 {page} 요청 실패: 상태 코드 {res.status_code}")
                break

            soup = BeautifulSoup(res.text, "html.parser")
            table = soup.find("table", class_="type2")
            if table is None:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                break

            df = pd.read_html(StringIO(str(table)), header=0)[0]
            df = df.dropna()

            # 👉 만약 페이지에 데이터가 없는 경우 자동 종료
            if df.empty:
                print(f"\n✅ 데이터 없음: 페이지 {page}에서 종료합니다.")
                break

            # 전일비 처리
            def parse_change(val):
                val = str(val).strip().replace(",", "")
                if val.startswith("상승"):
                    return int(val.replace("상승", "").strip())
                elif val.startswith("하락"):
                    return -int(val.replace("하락", "").strip())
                elif val.startswith("보합"):
                    return 0
                else:
                    return None

            df["전일비"] = df["전일비"].apply(parse_change)

            # 거래량 처리
            df["거래량"] = df["거래량"].astype(str).str.replace(",", "")
            df["거래량"] = df["거래량"].apply(lambda x: int(float(x)))

            # ✅ 출력 – 첫 페이지만 헤더 포함
            if page == 1:
                print(df.to_string(index=False))  # 헤더 포함
            else:
                print(df.to_string(index=False, header=False))  # 헤더 생략

            df_total = pd.concat([df_total, df], ignore_index=True)
            time.sleep(0.5)

        return df_total

    # 종목 코드 및 기업명 설정
    code = "077970"
    company_name = "STX엔진"

    # 데이터 수집
    df = get_day_price(code, max_pages=519)

    # 컬럼명 정리
    df.columns = ["날짜", "종가", "전일비", "시가", "고가", "저가", "거래량"]

    # 날짜 형식 변환
    df["날짜"] = pd.to_datetime(df["날짜"])

    # ✅ 날짜 기준으로 중복 제거
    df = df.drop_duplicates(subset="날짜", keep="first")

    # 날짜 내림차순 정렬
    df = df.sort_values("날짜", ascending=False)

    # CSV 파일로 저장
    filename = f"{company_name}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ {filename} 파일이 저장되었습니다.")


        날짜      종가  전일비      시가      고가      저가    거래량
2025.04.25 23600.0 1000 23000.0 23800.0 22650.0 349603
2025.04.24 22600.0  450 22200.0 22650.0 21800.0 219891
2025.04.23 22150.0  100 22400.0 22400.0 21950.0 111151
2025.04.22 22050.0  300 21300.0 22300.0 21300.0 127543
2025.04.21 21750.0 -850 22850.0 22950.0 21350.0 196246
2025.04.18 22600.0  300 22500.0 22650.0 21850.0 216707
2025.04.17 22300.0  400 21750.0 22500.0 21750.0 155407
2025.04.16 21900.0 -450 22350.0 22700.0 21800.0 148929
2025.04.15 22350.0  450 22300.0 22550.0 21850.0 225583
2025.04.14 21900.0  350 22150.0 22250.0 21550.0 197450
2025.04.11 21550.0  1200 21150.0 22200.0 21000.0 445500
2025.04.10 20350.0  1260 20300.0 20550.0 19900.0 164051
2025.04.09 19090.0  -140 19900.0 19900.0 18600.0 150903
2025.04.08 19230.0   920 18970.0 19600.0 18740.0 221205
2025.04.07 18310.0 -1890 19400.0 19400.0 18310.0 318589
2025.04.04 20200.0  -500 20300.0 20950.0 19900.0 218248
2025.04.03 20700.0   200 19900.0 21050.0 19710.0 170504
202